In [18]:
import os
os.makedirs("../data", exist_ok=True)


!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1330k  100 1330k    0     0  2600k      0 --:--:-- --:--:-- --:--:-- 2639k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1253k  100 1253k    0     0  3124k      0 --:--:-- --:--:-- --:--:-- 3182k


In [19]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [20]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [21]:
df_train = read_dataframe('..//data//green_tripdata_2024-01.parquet')
df_val = read_dataframe('..//data//green_tripdata_2024-02.parquet')

In [22]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [23]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [24]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [25]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/colome8/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=da87491f-4a08-4274-a3ca-a164f820d4fa&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=c27efea6e0d9559b14c718951a781b33fc28d42a28079e28189c2c06f5e3bd0a




Output()

Accessing as colome8

Initialized MLflow to track repo "colome8/nyc-taxi-time-prediction"

Repository colome8/nyc-taxi-time-prediction initialized!

https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow


2024/09/17 22:00:55 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/52f4281e23894b08be828e86a3d02bc9', creation_time=1726632055317, experiment_id='0', last_update_time=1726632055317, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [26]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [33]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("colome8/nyc-taxi-time-prediction")

# Upload file 1
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="..\data\green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

# Upload file 2
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="..\data\green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [34]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [35]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [36]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [37]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:5.34609                           
[1]	validation-rmse:5.26751                           
[2]	validation-rmse:5.27044                           
[3]	validation-rmse:5.27464                           
[4]	validation-rmse:5.27248                           
[5]	validation-rmse:5.27867                           
[6]	validation-rmse:5.27981                           
[7]	validation-rmse:5.28211                           
[8]	validation-rmse:5.28590                           
[9]	validation-rmse:5.28598                           
[10]	validation-rmse:5.28486                          
  0%|          | 0/10 [00:42<?, ?trial/s, best loss=?]

2024/09/17 22:06:06 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 22:06:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciadatos\nyc-taxi-time-prediction\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:06:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 22:06:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciad

[0]	validation-rmse:7.73286                                                      
[1]	validation-rmse:6.80590                                                      
[2]	validation-rmse:6.20216                                                      
[3]	validation-rmse:5.82279                                                      
[4]	validation-rmse:5.58962                                                      
[5]	validation-rmse:5.44818                                                      
[6]	validation-rmse:5.36637                                                      
[7]	validation-rmse:5.31839                                                      
[8]	validation-rmse:5.28832                                                      
[9]	validation-rmse:5.26849                                                      
[10]	validation-rmse:5.25598                                                     
[11]	validation-rmse:5.24521                                                     
[12]	validation-

2024/09/17 22:08:20 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 22:08:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciadatos\nyc-taxi-time-prediction\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:08:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 22:08:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

c:\Users\colom\OneDrive - ITESO\iteso\5to semestre\cienciad

[0]	validation-rmse:8.69612                                                      
[1]	validation-rmse:8.32449                                                     
[2]	validation-rmse:7.99131                                                     
[3]	validation-rmse:7.69611                                                     
[4]	validation-rmse:7.43241                                                     
[5]	validation-rmse:7.19975                                                     
[6]	validation-rmse:6.99362                                                     
[7]	validation-rmse:6.81262                                                     
[8]	validation-rmse:6.65337                                                     
[9]	validation-rmse:6.51264                                                     
[10]	validation-rmse:6.38776                                                    
[11]	validation-rmse:6.27940                                                    
[12]	validation-rmse:6.1839

...

2024/09/17 22:09:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run unequaled-mule-602 at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/265a791bcbbf47a2ab8a0fdb37e9f1d3.

2024/09/17 22:09:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0.



 30%|███       | 3/10 [04:27<10:23, 89.02s/trial, best loss: 5.199804099270815]

2024/09/17 22:09:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run Xgboost Hyper-parameter Optimization at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/06a2e657bba94fbc84d20e09ab79ece2.


2024/09/17 22:09:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0.


KeyboardInterrupt: 

In [ ]:
best_params

In [ ]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

In [ ]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

In [ ]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)